<a href="https://colab.research.google.com/github/blanca-32/Biocomputacional/blob/main/quiz2_blancac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quiz 2 Biología Computacional 2025-2
Blanca Sofia Cerinza - Asignatura Fundamentos de Biología computacional

Porfe los voy a dividir por puntos, espero mi trabajo sea de tu total agrado.

1. Crear carpeta y descargar archivo correspondiente al gen NRAMP6 (1 punto)

In [ ]:
import os

carpeta = "Genes"
os.makedirs(carpeta, exist_ok=True)

# Descargar archivo NRAMP5
!wget -O NRAMP5.fa "https://raw.githubusercontent.com/Pipe-95/TallerBiologiaComputacional/refs/heads/main/NRAMP5.fa"
archivo_destino = os.path.join(carpeta, "NRAMP5.fa")

print(f"Archivo descargado en: {archivo_destino}")


--2025-09-30 20:46:04--  https://raw.githubusercontent.com/Pipe-95/TallerBiologiaComputacional/refs/heads/main/NRAMP5.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5013 (4.9K) [text/plain]
Saving to: ‘NRAMP5.fa’

NRAMP5.fa           100%[===================>]   4.90K  --.-KB/s    in 0s      

2025-09-30 20:46:04 (29.7 MB/s) - ‘NRAMP5.fa’ saved [5013/5013]

Archivo descargado en: Genes/NRAMP5.fa


2. Descargar cromosoma 9 del genoma de Teobroma cacao de NCBI (2 puntos)

In [ ]:
# Instalar Biopython
!pip install biopython

from Bio import Entrez

Entrez.email = "bscerinzap@eafit.edu.co"

# Descargar la secuencia del cromosoma 8 de Theobroma cacao
handle = Entrez.efetch(
    db="nucleotide",
    id="NC_030857.1",
    rettype="fasta",
    retmode="text"
)

with open("Theobroma_cacao_chr8.fa", "w") as f:
    f.write(handle.read())
handle.close()

print("Archivo descargado: Theobroma_cacao_chr8.fa")

# Mostrar las primeras líneas del archivo para verificar
with open("Theobroma_cacao_chr8.fa") as f:
    for i in range(5):
        print(f.readline().strip())


Archivo descargado: Theobroma_cacao_chr8.fa
>NC_030857.1 Theobroma cacao cultivar B97-61/B2 chromosome 8, Criollo_cocoa_genome_V2, whole genome shotgun sequence
CGCCCCCCTCTTTTGTACTTTTCTATCGTTATTAATAAAATTTAACAGGGTAGCTGGGCCCTGCGTGGAC
TTCCAGTTTAAAAAAAAAAAAAAAACCCTAAACCCTAAACTCTAAACCCTAACCCCTAACCCCTAACCCC
TAAACCCTAACCCTTTAAACCCCAAACCCTTAAACCCTAAACCTTAACCCCTAACCCCTAACCCCAAACT
CCTGAACCCTGAACCCTGAACCCTGAACCCTACCCCTAAACCCTAAACCCTAAACCTAAACCTAAACCCT


3. Realizar un BLAST del gen descargado como query contra el cromosoma 9 y responder las siguientes preguntas (2 puntos)

In [ ]:
import pandas as pd

# Instalar BLAST+
!apt-get install ncbi-blast+ -y

!wget -O NRAMP5.fa "https://raw.githubusercontent.com/Pipe-95/TallerBiologiaComputacional/refs/heads/main/NRAMP5.fa"

!wget -O Theobroma_cacao_chr8.fa "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=NC_030857.1&db=nucleotide&report=fasta&retmode=text"

!makeblastdb -in Theobroma_cacao_chr8.fa -dbtype nucl -out cacao_chr8_db

!blastn -query NRAMP5.fa -db cacao_chr8_db -out resultados_blast_NRAMP5_chr8.txt -outfmt 6

cols = [
    "query_id","subject_id","identity","align_len","mismatches","gap_opens",
    "q_start","q_end","s_start","s_end","evalue","bit_score"
]

try:
    resultados = pd.read_csv("resultados_blast_NRAMP5_chr8.txt", sep="\t", names=cols)
except pd.errors.EmptyDataError:
    resultados = pd.DataFrame(columns=cols)

print("BLAST de NRAMP5 vs cromosoma 8:")

# Pregunta 1: Número de alineamientos
num_matches = len(resultados)
print(f"1) Número de alineamientos para NRAMP5: {num_matches}")

# Pregunta 2: Mismo locus o regiones distintas
if num_matches > 1:
    rango_inicio = resultados['s_start'].min()
    rango_final  = resultados['s_end'].max()
    dispersion   = rango_final - rango_inicio

    if dispersion < 10000:
        print("2) Los alineamientos pertenecen al MISMO locus (coordenadas cercanas).")
    else:
        print("2) Los alineamientos caen en regiones DISTINTAS del cromosoma 8.")
elif num_matches == 1:
    print("2) Solo hay un alineamiento, pertenece a un único locus.")
else:
    print("2) No hubo alineamientos, no se puede determinar locus.")

# Pregunta 3: Comparación con genoma de referencia
if num_matches > 0:
    identidad = resultados['identity'].mean()
    mismatches = resultados['mismatches'].sum()
    gaps = resultados['gap_opens'].sum()
    print(f"3) Identidad promedio: {identidad:.2f}%")
    print(f"   Mismatches: {mismatches}, Gaps: {gaps}")

    if identidad > 95 and mismatches == 0 and gaps == 0:
        print("Coinciden casi completamente, sin diferencias notorias.")
    else:
        print("Se observan SNPs, gaps o mismatches en la comparación.")
else:
    print("simplemente no hay alineamientos para comparar.")

# Pregunta 4: Sugerencia de primers
if num_matches > 0:
    mejor_hit = resultados.sort_values("bit_score", ascending=False).iloc[0]
    inicio = mejor_hit['s_start']
    fin    = mejor_hit['s_end']
    print(f"4) Para diseñar primers, escogería regiones conservadas en los extremos "
          f"del alineamiento principal: entre {min(inicio,fin)} y {max(inicio,fin)} "
          f"del cromosoma 8.")
else:
    print("4) No se puede sugerir posición de primers porque no hubo hits.")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ncbi-blast+ is already the newest version (2.12.0+ds-3build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
--2025-09-30 20:48:50--  https://raw.githubusercontent.com/Pipe-95/TallerBiologiaComputacional/refs/heads/main/NRAMP5.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5013 (4.9K) [text/plain]
Saving to: ‘NRAMP5.fa’

NRAMP5.fa           100%[===================>]   4.90K  --.-KB/s    in 0s      

2025-09-30 20:48:51 (59.8 MB/s) - ‘NRAMP5.fa’ saved [5013/5013]

--2025-09-30 20:48:51--  https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=NC_030857.1&db=nucleotide&report=fasta&retmode=text
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.go

El análisis del BLAST de la secuencia NRAMP5 contra el cromosoma 8 de Theobroma cacao mostró tres alineamientos significativos, distribuidos en diferentes regiones del cromosoma. Esto sugiere que existen copias en fragmentos homólogos dispersos, en lugar de un único locus.
La identidad promedio del 95.83%, junto con la presencia de 276 mismatches y 32 gaps, indica que aunque la secuencia conserva una alta similitud con el gen de referencia, se observan diferencias puntuales (SNP), por variación natural o divergencia evolutiva. Para el diseño de primers, sería mejor enfocarse en regiones conservadas y estables del alineamiento principal, particularmente en el intervalo comprendido entre las coordenadas 6,825,279 y 6,830,167 del cromosoma 8, donde la conservación de la secuencia es mayor y se asegura una amplificación específica.